In [1]:
from econml.grf import RegressionForest
from sklearn import clone
import os, pathlib
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import LogisticRegression

In [2]:
class t_learner:

    '''
    From Rosenberg Slides:
        µ^0(x) = E[Y (0) | X = x]
        µ^1(x) = E[Y (1) | X = x]
        τˆT (x) = µˆ1(x) − µˆ0(x)

    authors: Alene Rhea and Tamar Novetsky, April 1 2021
    '''

    def __init__(self, mu0_base, mu1_base):
        # Make copies of initialized base learners
        self.mu0_base = clone(mu0_base, safe=False)
        self.mu1_base = clone(mu1_base, safe=False)

    def fit(self, X, W, y):
        # Call fit methods on each base learner
        self.mu0_base.fit(X[W==0], y[W==0])
        self.mu1_base.fit(X[W==1], y[W==1])

    def predict(self, X):
        y1_preds = self.mu1_base.predict(X)
        y0_preds = self.mu0_base.predict(X)
        tau_preds = y1_preds - y0_preds
        return tau_preds.flatten()

In [3]:
class s_learner:

    '''
    From Rosenberg Slides:
        µ^(x,w) = E[Y | X = x, W = w]
        τˆS (x) = µˆ(x,1) − µˆ(x,0)

    authors: Kelsey Markey and Lauren D'Arinzo, April 4 2021
    '''

    def __init__(self, mu_base):
        # Make copies of initialized base learner
        self.mu_base = clone(mu_base, safe=False)

    def fit(self, X_W, y):
        # Call fit method
        self.mu_base.fit(X_W, y)

    def predict(self, X):
        X_W_1 = X.copy(deep=True)
        X_W_1["W"] = pd.Series(np.ones(len(X_W_1)))
        y1_preds = self.mu_base.predict(X_W_1)

        X_W_0 = X.copy(deep=True)
        X_W_0["W"] = pd.Series(np.zeros(len(X_W_0)))
        y0_preds = self.mu_base.predict(X_W_0)

        tau_preds = y1_preds - y0_preds
        return tau_preds.flatten()

In [4]:
class x_learner:

    '''
    From Rosenberg Slides:
        µ^0(x) = E[Y (0) | X = x]
        µ^1(x) = E[Y (1) | X = x]
        τˆx (x) = g(x)t_0ˆ(x) − (1-g(x))t_1ˆ(x)

    authors: Kelsey Markey and Lauren D'Arinzo, April 4 2021
    '''

    def __init__(self, mu0_base, mu1_base, tau0_base, tau1_base):
        # Make copies of initialized base learner
        self.mu0_base = clone(mu0_base, safe=False)
        self.mu1_base = clone(mu1_base, safe=False)
        self.tau0_base = clone(tau0_base, safe=False)
        self.tau1_base = clone(tau1_base, safe=False)

    def fit(self, X, W, y, fit_g):
        '''
        fit_g : boolean indicator if g should be fit to training data. if false, g must be passed explicitlly to x_learner.predict()
        '''
        # Call fit method
        self.mu0_base.fit(X[W==0], y[W==0])
        self.mu1_base.fit(X[W==1], y[W==1])
    
        #Impute y0 for treated group using mu0
        y0_treat=self.mu0_base.predict(X[W==1]).flatten()
        imputed_TE_treatment = y[W==1] - y0_treat

        #Impute y1 for control group using mu1
        y1_control=self.mu1_base.predict(X[W==0]).flatten()
        imputed_TE_control = y1_control - y[W==0] 

        #Fit tau0: CATE estimate fit to the control group
        self.tau0_base.fit(X[W==0], imputed_TE_control)

        #Fit tau1: CATE estimate fit to the treatment group
        self.tau1_base.fit(X[W==1], imputed_TE_treatment)

        if fit_g:
            print('X Learner with g fitted')
            #predict propensities from empirical data
            self.g_fit = LogisticRegression(fit_intercept=True, max_iter=2000).fit(
            X=X, y=W)
        
        else:
            print('X Learner with true propensities')
            self.g_fit = None


    def predict(self, X, g):
        '''
        g : weight vector that should be length of the test set. can be passed as None if g was fit to data
        '''
        tau0_preds = self.tau0_base.predict(X).flatten()
        tau1_preds = self.tau1_base.predict(X).flatten()
    
    
        if self.g_fit == None:
            tau_preds = (g.T * tau0_preds) + ((1-g).T*tau1_preds)
        else:
            g_preds = self.g_fit.predict_proba(X)[:, 1]
            tau_preds = (g_preds.T * tau0_preds) + ((1-g_preds).T*tau1_preds)

        # idea: allow g to be either be a vector or function?
        # if function: think about sklearn inputs (.predict) vs lamba functions (g(x))
        # if g_type == 'Function':
        #    g_preds = g(X)
        #    tau_preds = (g_preds.T * tau0_preds) + ((1-g_preds).T*tau1_preds)

        return tau_preds.flatten()

In [5]:
def fit_get_mse_t(train, test, mu0_base, mu1_base):
    '''
    mu0_base: base learner that has already been initialized
    mu1_base: base learner that has already been initialized
    '''
    #data preprocessing
    X_train = train.drop(columns=['treatment', 'Y', 'tau', 'pscore'])
    y_train = train['Y']
    W_train = train['treatment']
    X_test = test.drop(columns=['treatment', 'Y', 'tau', 'pscore'])

    #initialize metalearner
    T = t_learner(mu0_base=mu0_base, mu1_base=mu1_base)
    T.fit(X=X_train, W=W_train, y=y_train)
    
    # Predict test-set CATEs
    tau_preds = T.predict(X=X_test)

    # Calculate MSE on test set
    mse = np.mean((tau_preds - test.tau)**2)
    return mse

In [6]:
def fit_get_mse_s(train, test, mu_base):
    '''
    mu: base learner that has already been initialized
    '''
    #data preprocessing
    X_train = train.drop(columns=['treatment', 'Y', 'tau', 'pscore'])
    y_train = train['Y']
    W_train = train['treatment']
    X_test = test.drop(columns=['treatment', 'Y', 'tau', 'pscore'])

    #initialize metalearner
    S = s_learner(mu_base=mu_base)
    
    #fit S-learner
    X_W = pd.concat([X_train, W_train], axis=1)
    S.fit(X_W=X_W, y=y_train)
    
    # Predict test-set CATEs
    tau_preds = S.predict(X=X_test)

    # Calculate MSE on test set
    mse = np.mean((tau_preds - test.tau)**2)
    return mse

In [7]:
def main():
    # Set root directory
    # base_repo_dir = pathlib.Path(os.path.realpath(__file__)).parents[0]
    base_repo_dir = os.getcwd()
    
    #read in tuned hyperparameter files
    rf_t = json.load(open(base_repo_dir + '/configurations/hyperparameters/' + 'rf_t.json'))
    rf_s = json.load(open(base_repo_dir + '/configurations/hyperparameters/' + 'rf_s.json'))
    
    #rf_params = {'T': rf_T, 'S': rf_S, 'X': rf_X}
    rf_params = {'T': rf_t, 'S': rf_s}
    
    #read in base learner model types for each metalearner
    meta_base_dict = json.load(open(base_repo_dir + '/configurations/base_learners/' + 'base_learners.json'))
    
    for train_size in [5000, 10000]: #, 20000, 100000, 300000
        print('---------------------------')
        print('Training set size:', train_size)
        for sim in ['simA', 'simB', 'simC', 'simD', 'simE', 'simF']:
            print('---------------------------')
            print('Starting '+ sim)
            for i in range(3):
                print('')
                train = pd.read_parquet(base_repo_dir + '/data/' + str(train_size) + '/' + sim + 
                                        '/samp'+str(i+1)+'_train.parquet')
                test = pd.read_parquet(base_repo_dir + '/data/' + str(train_size) + '/' + sim +  
                                       '/samp'+str(i+1)+'_test.parquet')
                for metalearner in meta_base_dict.keys():

                    if metalearner == 'T':
                    # below logic needs to be generalized for other metalearners
                        for base_learner_dict in meta_base_dict[metalearner]:
                            print(sim+'/' +'sample_'+str(i+1)+'/'+metalearner+'-learner:')
                            if base_learner_dict['mu_0'] == 'rf':
                                mu0_hyperparams = rf_params[metalearner]['mu_0'][sim]
                                mu0_base = RegressionForest(honest=True, random_state=42, **mu0_hyperparams)
                            if base_learner_dict['mu_1'] == 'rf':
                                mu1_hyperparams = rf_params[metalearner]['mu_1'][sim]
                                mu1_base = RegressionForest(honest=True, random_state=42, **mu1_hyperparams)
                            mse = fit_get_mse_t(train, test, mu0_base, mu1_base)
                            print('     MSE=', mse)

                    if metalearner == 'S':
                        for base_learner_dict in meta_base_dict[metalearner]:
                            print(sim+'/' +'sample_'+str(i+1)+'/'+ metalearner+'-learner:')
                            if base_learner_dict['mu'] == 'rf':
                                mu_hyperparams = rf_params[metalearner]['mu'][sim]
                                mu_base = RegressionForest(honest=True, random_state=42, **mu_hyperparams)
                            mse = fit_get_mse_s(train, test, mu_base)
                            print('     MSE=', mse)


    return

In [8]:
main()

---------------------------
Training set size: 5000
---------------------------
Starting simA

simA/sample_1/T-learner:
     MSE= 88.63872206863563
simA/sample_1/S-learner:
     MSE= 29.504

simA/sample_2/T-learner:
     MSE= 129.4262566100085
simA/sample_2/S-learner:
     MSE= 29.36192

simA/sample_3/T-learner:
     MSE= 95.17221816324512
simA/sample_3/S-learner:
     MSE= 29.50784
---------------------------
Starting simB

simB/sample_1/T-learner:
     MSE= 6775.965620493603
simB/sample_1/S-learner:
     MSE= 3662.7435690545954

simB/sample_2/T-learner:
     MSE= 5400.17845111733
simB/sample_2/S-learner:
     MSE= 2222.1920170910043

simB/sample_3/T-learner:
     MSE= 2883.7380421634894
simB/sample_3/S-learner:
     MSE= 1637.1524024289147
---------------------------
Starting simC

simC/sample_1/T-learner:
     MSE= 0.044908067239896925
simC/sample_1/S-learner:
     MSE= 0.047138779011675826

simC/sample_2/T-learner:
     MSE= 0.045876581434470036
simC/sample_2/S-learner:
     MSE= 0